# **train.ipynb**

본 파일에서는, **전처리한 네이버 영화 리뷰 데이터셋을 활용하여 우리가 생성한 BertModel을 학습**시켜볼 것이다. 이후, **최종 성능(테스트 정확도)까지 출력**해볼 것이다.



In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/NLP

/content/gdrive/MyDrive/NLP


연습 문제를 시작하기에 앞서, import_ipynb를 설치한 후 import 하여 train.ipynb에서 필요한 preproc.ipynb, model.ipynb 내 함수를 호출한다.  
또한, 필요한 라이브러리를 호출한다.

In [3]:
!pip install import_ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import import_ipynb

from Preproc import preproc
from model import get_model, get_model_with_params, BertModelInitialization
import random
import numpy as np
import pandas as pd
import torch
from tqdm.notebook import tqdm
import time

importing Jupyter notebook from Preproc.ipynb
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 1.8 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 11.1 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595753 sha256=7ddd0018407d54e2a6cd7a0695fb36e18b57e64e6a5c6f42ddaadc071415df6c
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 39.8 MB/s 
Looking in indexes

모델의 예측 정확도를 산출하는 함수인 **accuracy**를 정의하자. 해당 함수는 학습한 모델의 validation 점수와 test의 결과를 계산할 때 사용된다.


In [5]:
# 정확도 계산 함수
def accuracy(preds, labels):
    f_pred = np.argmax(preds, axis=1).flatten()
    f_labels = labels.flatten()
    return np.sum(f_pred == f_labels) / len(f_labels)

**잠깐 ✔ 랜덤시드 고정이란 무엇인가?**
> 학습된 모델의 결과를 동일하게 재현(Reproduction)하는 것은 여러가지 상황에서 팔요하다.  
> 모델을 돌릴 때마다 결과가 달라지지 않도록 고정하는 것이다.  

- 수상자가 되어 코드의 정합성을 검증 받게 될 경우,

- 경진대회 참가 도중 팀을 이루어 결과를 공유해야 되는 경우,

- 논문을 작성하여 그 결과를 Reproduction 해야하는 경우 등 여러 상황에서 필요하다.  

- 본 중간 미션 대회 역시, (1) preproc.ipynb 내 섹터 별 자동 점수 반환 및 (2) 최종 평가 과정에서 혼동을 방지하기 위하여 랜덤시드를 고정해야 한다. 주어진 2022 시드 값을 절대 수정하지 않도록 하자.

참고 자료:
https://dacon.io/codeshare/2363
https://pytorch.org/docs/stable/notes/randomness.html


In [6]:
# 재현을 위해 랜덤시드 고정
seed_val = 2022

In [7]:
# 랜덤하게 데이터를 추출하기 위한 seed 값 설정
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

학습에 활용될 데이터셋 및 토크나이저를 지정하자. 이후, 데이터셋을 전처리하여 train, validation, test 각각의 데이터로더에 입력하자.


In [8]:
from tokenization import KoBertTokenizer

whole_dataset = pd.read_csv('ratings.txt', delimiter="\t") # 전체 데이터를 불러오기
tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert") # KoBERTTokenizer를 불러오기

train_dataloader, validation_dataloader, test_dataloader = preproc(tokenizer, whole_dataset)

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

length of whole_data : 200000
length of train_data : 180000


Preproc.ipynb:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


BertModel을 생성하여 GPU 혹은 CPU에 등록하자.
- 이때 BertModelInitialization()를 실행할 경우 기존 Device에 등록된 BertModel은 초기화되니, 한 번만 실행한 이후로는 사용하지 않도록 유의하여 사용해야 한다.
- 디바이스를 설정하자.
- 본격적인 학습에 앞서 train에 대한 model, 옵티마이저, 스케줄러, 에폭을 지정하고, 모델의 그래디언트를 초기화하자.

In [9]:
# 기존 Device에 등록된 BertModel은 초기화되니, 유의하여 사용할 것.
# 한 번만 실행하고, 그 이후로는 사용하지 않도록 조심!
BertModelInitialization()

Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/369M [00:00<?, ?B/s]

You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# GPU 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model, optimizer, scheduler, epochs = get_model_with_params(len(train_dataloader), device)

# 그래디언트 초기화
model.zero_grad()

You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**train 및 validation**  
본격적으로 학습을 진행해보자. epoch 만큼 학습 loop를 반복할 것이다.
- 우리가 생성한 model에, 배치 데이터에 대한 input_ids, attention_mask, labels 변수를 입력하여 순전파를 진행할 것이다.
- 이후 역전파 과정을 통해 매개변수가 조절되며 학습이 이루어진다.
- 한 차례 학습이 이루어질 때마다 average training loss 및 validation 정확도를 출력할 것이다.
- 학습이 완료된 모델을 특정 경로(PATH)에 저장할 것이다.


In [11]:
# 에폭만큼 반복
for epoch_i in range(epochs):
    print("")
    print('========{:}번째 Epoch / 전체 {:}회 ========'.format(epoch_i + 1, epochs))
    print('훈련 중')

    total_loss = 0 # 로스 초기화 
    sum_loss = 0
    model.train()  # 훈련모드로 변경
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(tqdm(train_dataloader)):
        
        if step % 50 == 0:
          print("{}번째 까지의 평균 loss : {}".format(step, sum_loss/50))
          sum_loss = 0

        batch = tuple(t.to(device) for t in batch)   # 배치를 GPU에 넣음
        b_input_ids, b_input_mask, b_labels = batch  # 배치에서 데이터 추출

        # model.py에서 model 함수를 정의할 때 BertForSequenceClassification를 활용하였다. 
        # 여기서 BertForSequenceClassification는 input_ids, attention_mask, labels 변수를 입력받는 'forward' 함수를 내장한다.
        # forward 함수는 forward(self, input_ids, attention_mask, token_type_ids, position_ids, head_mask, inputs_embeds, labels, output_attentions, output_hidden_states)와 같은 함수 파라미터를 갖는다.
        # model 함수를 통해 forward를 수행하기 위해 우리가 입력해야 하는 변수는 input_ids, attention_mask, labels 이다.
        # 위의 코드에서 정의한 배치 데이터를 model 함수에 입력하여, 배치에 대한 Forward를 수행해보자.                 
        
        ##여기에 코드 작성
        outputs = model.forward(input_ids = b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        
        loss = outputs[0]
        total_loss += loss.item() # 총 로스 계산
        sum_loss += loss.item()

        loss.backward() # Backward 수행으로 그래디언트 계산
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # 그래디언트 클리핑
        optimizer.step() # 그래디언트를 통해 가중치 파라미터 업데이트
        scheduler.step()  # 스케줄러로 학습률 감소
        model.zero_grad() # 그래디언트 초기화

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    
    #### 검증 ####
    
    print("")
    print("검증 중")

    model.eval()#검증모드

    # 변수 초기화
    eval_accuracy = 0
    nb_eval_steps = 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            attention_mask=b_input_mask)
        
        # 결과 값 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))

# 학습된 모델을 해당 PATH에 저장
PATH = "model.pt"
torch.save(model.state_dict(), PATH)

print("")
print("Training complete!")


========1번째 Epoch / 전체 3회 ========
훈련 중


  0%|          | 0/5063 [00:00<?, ?it/s]

0번째 까지의 평균 loss : 0.0
50번째 까지의 평균 loss : 0.6929739058017731
100번째 까지의 평균 loss : 0.564224191904068
150번째 까지의 평균 loss : 0.4966108626127243
200번째 까지의 평균 loss : 0.44774107694625853
250번째 까지의 평균 loss : 0.436091895699501
300번째 까지의 평균 loss : 0.448324091732502
350번째 까지의 평균 loss : 0.4030429241061211
400번째 까지의 평균 loss : 0.3585509303212166
450번째 까지의 평균 loss : 0.35419508576393127
500번째 까지의 평균 loss : 0.3712117278575897
550번째 까지의 평균 loss : 0.3840119826793671
600번째 까지의 평균 loss : 0.3500893349945545
650번째 까지의 평균 loss : 0.37336898416280745
700번째 까지의 평균 loss : 0.36723428159952165
750번째 까지의 평균 loss : 0.3647089910507202
800번째 까지의 평균 loss : 0.3438014271855354
850번째 까지의 평균 loss : 0.34383672326803205
900번째 까지의 평균 loss : 0.33338602125644684
950번째 까지의 평균 loss : 0.3585482254624367
1000번째 까지의 평균 loss : 0.3559280240535736
1050번째 까지의 평균 loss : 0.32143055349588395
1100번째 까지의 평균 loss : 0.31732676714658736
1150번째 까지의 평균 loss : 0.32618727460503577
1200번째 까지의 평균 loss : 0.3368974584341049
1250번째 까지의 평균 loss : 0.341495043

  0%|          | 0/5063 [00:00<?, ?it/s]

0번째 까지의 평균 loss : 0.0
50번째 까지의 평균 loss : 0.21025345385074615
100번째 까지의 평균 loss : 0.21017018731683493
150번째 까지의 평균 loss : 0.20433478999882937
200번째 까지의 평균 loss : 0.22658727247267962
250번째 까지의 평균 loss : 0.21547046966850758
300번째 까지의 평균 loss : 0.20258600659668446
350번째 까지의 평균 loss : 0.23869008496403693
400번째 까지의 평균 loss : 0.22572362020611764
450번째 까지의 평균 loss : 0.22187885999679566
500번째 까지의 평균 loss : 0.21556238308548928
550번째 까지의 평균 loss : 0.23258173778653146
600번째 까지의 평균 loss : 0.2183183019608259
650번째 까지의 평균 loss : 0.20559594109654428
700번째 까지의 평균 loss : 0.20394268073141575
750번째 까지의 평균 loss : 0.1868498605489731
800번째 까지의 평균 loss : 0.21621844224631787
850번째 까지의 평균 loss : 0.21632259093225004
900번째 까지의 평균 loss : 0.2278812078386545
950번째 까지의 평균 loss : 0.2128147465363145
1000번째 까지의 평균 loss : 0.20257090717554094
1050번째 까지의 평균 loss : 0.21027284257113935
1100번째 까지의 평균 loss : 0.2230704339966178
1150번째 까지의 평균 loss : 0.22633169308304787
1200번째 까지의 평균 loss : 0.21177577875554562
1250번째 까지의 평균 loss 

  0%|          | 0/5063 [00:00<?, ?it/s]

0번째 까지의 평균 loss : 0.0
50번째 까지의 평균 loss : 0.12754891220480202
100번째 까지의 평균 loss : 0.15110843926668166
150번째 까지의 평균 loss : 0.1454973492771387
200번째 까지의 평균 loss : 0.1383903439901769
250번째 까지의 평균 loss : 0.14035287853330375
300번째 까지의 평균 loss : 0.1638853844255209
350번째 까지의 평균 loss : 0.11850619312375783
400번째 까지의 평균 loss : 0.15073652260005474
450번째 까지의 평균 loss : 0.15639678860083223
500번째 까지의 평균 loss : 0.1323100594058633
550번째 까지의 평균 loss : 0.13752451227977872
600번째 까지의 평균 loss : 0.1152773105725646
650번째 까지의 평균 loss : 0.1689569691941142
700번째 까지의 평균 loss : 0.13465088102966546
750번째 까지의 평균 loss : 0.1429837654158473
800번째 까지의 평균 loss : 0.13801050685346128
850번째 까지의 평균 loss : 0.1519531973078847
900번째 까지의 평균 loss : 0.153631204739213
950번째 까지의 평균 loss : 0.1367013043165207
1000번째 까지의 평균 loss : 0.1455419347807765
1050번째 까지의 평균 loss : 0.1280919370613992
1100번째 까지의 평균 loss : 0.14515168663114308
1150번째 까지의 평균 loss : 0.11463894525542856
1200번째 까지의 평균 loss : 0.15813915837556125
1250번째 까지의 평균 loss : 0.1387

**test**  
학습된 모델에 test용 데이터를 입력하여 test 결과를 출력하자.
- test용 데이터로더를 활용하여 배치 데이터에 대한 input_ids, mask를 model에 입력하여 순전파를 실행하고, 실행 결과를 outputs에 저장하자.
- 이때 test는 '학습'이 목적이 아니므로 그래디언트를 계산하지 않도록 한다.
- outputs와 실제 정답인 label_ids를 비교하여 모델의 최종 test accuracy를 확인하자. 모델이 88%의 정확도를 넘기는가?


In [12]:
print("")
print("테스트 중")

model.eval()

# 변수 초기화
eval_accuracy = 0
nb_eval_steps = 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for batch in test_dataloader:
  # 배치를 GPU에 넣음
  batch = tuple(t.to(device) for t in batch)
        
  # 배치에서 데이터 추출
  b_input_ids, b_input_mask, b_labels = batch
        
  # 그래디언트 계산 안함
  with torch.no_grad():     
      # Forward 수행
      outputs = model(b_input_ids, 
                      attention_mask=b_input_mask)
        
  # 로스 구함
  logits = outputs[0]

  # CPU로 데이터 이동
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # 출력 로짓과 라벨을 비교하여 정확도 계산
  tmp_eval_accuracy = accuracy(logits, label_ids)
  eval_accuracy += tmp_eval_accuracy
  nb_eval_steps += 1

print("")
print("  Accuracy: {0:.4f}".format(eval_accuracy/nb_eval_steps))


테스트 중

  Accuracy: 0.9023
